# Regular Expression Dataset Generation

## Utilities
### Definitions

In [3]:
import re
from itertools import product
from itertools import islice
from string import ascii_letters
import csv
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import keras
from keras import layers
from keras.preprocessing import sequence

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding

/Users/ischeinfeld/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
def all_strings(max_length, alphabet):
    """Returns a generator of all strings up to a given length from an alphabet"""
    for length in range(max_length + 1):
        for s in product(alphabet, repeat=length):
            yield ''.join(s)

In [5]:
def extract_valid_regexes(candidates):
    """Takes a generator of strings and returns a generator of compiled valid regexes"""
    for c in candidates:
        try:
            yield re.compile(c)
        except:
            pass

In [6]:
def all_regexes(max_length, alphabet):
    """Returns a generator of all valid regexes up to a length from an alphabet"""
    return extract_valid_regexes(all_strings(max_length, alphabet))

## Dataset Generation Utilities
### Definitions

In [7]:
def regex_apply_all(regex, max_length, alphabet):
    """Returns a generator that takes a compiled regex and yields
    tuples for every string of length from alphabet with whether
    or not it matches the regex"""
    for string in all_strings(max_length, alphabet):
        yield (string, bool(regex.fullmatch(string)))

In [8]:
def triple_count_bound(regex_max_length, string_max_length, alphabet, regex_chars):
    """Returns an upper bound on the size of a dataset generated with the same parameters"""
    count_regexes = sum(len(alphabet + regex_chars) ** length for length in range(regex_max_length + 1))
    count_strings = sum(len(alphabet) ** length for length in range(string_max_length + 1))
    return count_regexes * count_strings

In [9]:
def triple_generator(regex_max_length, string_max_length, alphabet, regex_chars):
    """Returns a generator that gives for every regex-string pair
    up to a length whether or not they match"""
    for regex in all_regexes(regex_max_length, alphabet + regex_chars):
        for string in all_strings(string_max_length, alphabet):
            yield regex.pattern, string, bool(regex.fullmatch(string))

In [23]:
def dataset_generator(regex_max_length, string_max_length, alphabet, regex_chars):
    """Returns a generator that gives for every regex-string pair
    up to a length whether or not they match, with strings encoded
    as character index lists"""
    regex_char_int = {c: i + 1 for i, c in enumerate(alphabet + regex_chars)}
    string_char_int = {c: i + 1 for i, c in enumerate(alphabet)}
    
    for regex in all_regexes(regex_max_length, alphabet + regex_chars):
        regex_ints = [regex_char_int[c] for c in regex.pattern]
        for string in all_strings(string_max_length, alphabet):
            string_ints = [string_char_int[c] for c in string]
            if len(regex.pattern) != 0 and len(string) != 0:
                yield regex_ints, string_ints, np.int64(int(bool(regex.fullmatch(string))))

### Examples

In [24]:
list(islice(dataset_generator(2, 2, "a", "*()"), 5))

Exception ignored in: <generator object extract_valid_regexes at 0x11e317410>
RuntimeError: generator ignored GeneratorExit


[([1], [1], 1),
 ([1], [1, 1], 0),
 ([1, 1], [1], 0),
 ([1, 1], [1, 1], 1),
 ([1, 2], [1], 1)]

## Generate Dataset

### Generate, Balance, and Split

In [50]:
# Generate
alphabet = "abc"
regex_chars = "|*()"
regex_max_len = 4
string_max_len = 6
Xr, Xs, y = map(np.array, zip(*dataset_generator(regex_max_len, string_max_len,
                                                   alphabet, regex_chars)))

Xr, Xs, y = shuffle(Xr, Xs, y, random_state=42)
print("Initial data size: {}".format(Xr.shape))

# Balance
pos = (y == 1)
neg = (y == 0)
class_size = min(pos.sum(), len(pos) - pos.sum())
Xr_pos, Xs_pos, y_pos = Xr[pos][:class_size], Xs[pos][:class_size], y[pos][:class_size]
Xr_neg, Xs_neg, y_neg = Xr[neg][:class_size], Xs[neg][:class_size], y[neg][:class_size]

Xr = np.concatenate((Xr_pos, Xr_neg))
Xs = np.concatenate((Xs_pos, Xs_neg))
y = np.concatenate((y_pos, y_neg))

Xr, Xs, y = shuffle(Xr, Xs, y, random_state=42)

print("Balanced data size: {}".format(Xr.shape))

# Split
Xr_train, Xr_test, Xs_train, Xs_test, y_train, y_test = train_test_split(Xr, Xs, y, test_size=0.33, random_state=42)

print("Train/Test split data size: {}/{}".format(Xr_train.shape, Xr_test.shape))

Initial data size: (713076,)
Balanced data size: (3456,)
Train/Test split data size: (2315,)/(1141,)


### Prepare

In [51]:
# Padding #TODO is this necessary?
Xr_train = sequence.pad_sequences(Xr_train, maxlen=regex_max_len)
Xs_train = sequence.pad_sequences(Xs_train, maxlen=string_max_len)

Xr_test = sequence.pad_sequences(Xr_test, maxlen=regex_max_len)
Xs_test = sequence.pad_sequences(Xs_test, maxlen=string_max_len)

regex_embedding_input_dim = len(alphabet + regex_chars) + 1
string_embedding_input_dim = len(alphabet) + 1

## LSTM

In [52]:
# build the model
regex_input = Input(shape=(regex_max_len,))
regex_embedding = Embedding(regex_embedding_input_dim, 4, input_length=regex_max_len)(regex_input)
regex_vector = LSTM(50)(regex_embedding)

string_input = Input(shape=(string_max_len,))
string_embedding = Embedding(string_embedding_input_dim, 4, input_length=string_max_len)(string_input)
string_vector = LSTM(50)(string_embedding)

x = layers.concatenate([regex_vector, string_vector])
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid', name='predictions')(x)

model = Model(inputs=[regex_input, string_input], outputs=predictions)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model.summary())
model.fit([Xr_train, Xs_train], y_train, validation_data=([Xr_test, Xs_test], y_test), epochs=100, batch_size=64)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           (None, 4)            0                                            
__________________________________________________________________________________________________
input_25 (InputLayer)           (None, 6)            0                                            
__________________________________________________________________________________________________
embedding_23 (Embedding)        (None, 4, 4)         32          input_24[0][0]                   
__________________________________________________________________________________________________
embedding_24 (Embedding)        (None, 6, 4)         16          input_25[0][0]                   
__________________________________________________________________________________________________
lstm_23 (L

In [53]:
# Final evaluation of the model
scores = model.evaluate([Xr_test, Xs_test], y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 90.71%
